# Tensor Methods Homework 2


### 1 Forming the multiplication tensor "on paper"

$$\begin{pmatrix} a_1 & a_3 \\ a_2 & a_4 \end{pmatrix} \cdot \begin{pmatrix} b_1 & b_3 \\ b_2 & b_4 \end{pmatrix} = \begin{pmatrix} a_1 b_1 + a_3 b_2 & a_1 b_3 + a_3 b_4 \\ a_2 b_1 + a_4 b_2 & a_2 b_3 + a_4 b_4 \end{pmatrix} = \begin{pmatrix} c_1 & c_3 \\ c_2 & c_4 \end{pmatrix}$$

Find $T \in \mathbb{R}^{4 \times 4 \times 4}$ such that $c_k = \sum_{i=1}^4 \sum_{j=1}^4 t_{ijk} a_i b_j$.

Using the naive implementation from above we get

$$T = \left[\begin{array}{cccc|cccc|cccc|cccc}
  1 &   & &   &     &   & &   &   & 1 &   &   &   & &   &   \\
    &   & &   &  1 &   & &    &   &   &   &   &   & & 1 &   \\
    & 1 & &   &     &   & &   &   &   & 1 &   &   & &   &   \\
    &   & &   &     & 1 & &   &   &   &   &   &   & &   & 1 \\
\end{array}\right]$$

### 2 Forming a CPD of the multiplication tensor "on paper"

Find $U, V, W \in \mathbb{R}$ such that $T = \sum_{\alpha=1}^8 u_\alpha \otimes v_\alpha \otimes w_\alpha$.

Choosing $U, V, W$ such that $u_\alpha \otimes v_\alpha \otimes w_\alpha = e_i \otimes e_j \otimes e_k$ for each $t_{ijk} = 1$ yields

$$ U = \begin{pmatrix}
    1 &   &   &   & 1 &   &   &   \\
      &   & 1 &   &   &   & 1 &   \\
      & 1 &   &   &   & 1 &   &   \\
      &   &   & 1 &   &   &   & 1 \\
\end{pmatrix}$$

$$ V = \begin{pmatrix}
    1 &   & 1 &   &   &   &   &   \\
      & 1 &   & 1 &   &   &   &   \\
      &   &   &   & 1 &   & 1 &   \\
      &   &   &   &   & 1 &   & 1 \\
\end{pmatrix}$$

$$ W = \begin{pmatrix}
    1 & 1 &   &   &   &   &   &   \\
      &   & 1 & 1 &   &   &   &   \\
      &   &   &   & 1 & 1 &   &   \\
      &   &   &   &   &   & 1 & 1 \\
\end{pmatrix}$$

### 3 Implementing mode contraction

In [1]:
function contract(S::AbstractArray, Z::AbstractMatrix, k::Integer)
    # check arguments
    d = ndims(S)
    n = size(S)
    r, nₖ = size(Z)
    if !(0 < k <= d)
        throw(ArgumentError("Contraction index is not in range 1 ≤ k ≤ $d: got $k"))
    end
    if nₖ != n[k]
        throw(ArgumentError("Dimension of matrix does not match tensor: expected $(n[k]), got $nₖ"))
    end
    
    # initialize new array
    T = promote_type(eltype(S), eltype(Z))
    Rs = Base.setindex(n, r, k)
    A = similar(S, T, Rs)

    # contract
    for α = 1:r
        for I in CartesianIndices(S)
            J = Base.setindex(I, α, k)
            A[J] += S[I] * Z[α,I[k]]
        end
    end

    return A
end

contract (generic function with 1 method)

In [2]:
S = rand(3,5,2)
Z = rand(4,5)
k = 2

contract(S,Z,k)

3×4×2 Array{Float64, 3}:
[:, :, 1] =
 1.28161  1.1103    1.75035   0.925998
 1.35912  0.8513    1.2198    0.778232
 1.04121  0.642308  0.804027  0.60413

[:, :, 2] =
 1.88474  1.21508   1.84602  1.09282
 1.69675  1.19424   1.71891  1.06014
 1.44863  0.792002  1.25659  0.814927

### 4 Evaluating a CPD

In [3]:
function evalCPD(Us::Vararg{AbstractMatrix,N}) where {N}
    # check arguments
    if N == 0
        throw(ArgumentError("No arguments provided."))
    end
    n = Tuple([size(U, 1) for U in Us])
    r = size(Us[1], 2)
    T = eltype(Us[1])
    for U in Us
        T = promote_type(T, eltype(U))
        rk = size(U, 2)
        if r != rk
            throw(ArgumentError("Matrices do not have matching rank dimensions."))
        end
    end

    # initialize tensor
    S = zeros(T, n)

    # evaluate CPD
    for I in CartesianIndices(S)
        S[I] = sum([prod([Us[k][I[k],α] for k = 1:length(Us)]) for α = 1:r])
    end

    return S
end

evalCPD (generic function with 1 method)

In [4]:
U1 = rand(5, 8)
U2 = rand(4, 8)
U3 = rand(3, 8)

evalCPD(U1, U2, U3)

5×4×3 Array{Float64, 3}:
[:, :, 1] =
 0.889022  0.728917  0.561712  0.853951
 0.749597  0.883557  1.0095    1.2401
 0.727435  1.02099   1.26343   0.931078
 1.60519   1.55702   1.48317   1.64987
 1.27726   1.51711   1.70824   1.79924

[:, :, 2] =
 0.762982  0.623928  0.459752  0.545446
 0.487429  0.549795  0.659175  0.771766
 0.635766  0.76452   0.903463  0.574327
 1.28676   1.1626    1.11641   1.06603
 1.18815   1.23337   1.28494   1.23375

[:, :, 3] =
 0.686904  0.633191  0.558383  0.789021
 0.421195  0.520284  0.508647  0.64476
 0.595308  0.985194  1.20883   0.81159
 1.33806   1.51049   1.53872   1.55
 0.845663  1.06218   1.17339   1.24037

### 5 Implementing the multiplication tensor

In [5]:
U = [1 0 0 0 1 0 0 0; 0 0 1 0 0 0 1 0; 0 1 0 0 0 1 0 0; 0 0 0 1 0 0 0 1]
V = [1 0 1 0 0 0 0 0; 0 1 0 1 0 0 0 0; 0 0 0 0 1 0 1 0; 0 0 0 0 0 1 0 1]
W = [1 1 0 0 0 0 0 0; 0 0 1 1 0 0 0 0; 0 0 0 0 1 1 0 0; 0 0 0 0 0 0 1 1]

T = evalCPD(U, V, W)

4×4×4 Array{Int64, 3}:
[:, :, 1] =
 1  0  0  0
 0  0  0  0
 0  1  0  0
 0  0  0  0

[:, :, 2] =
 0  0  0  0
 1  0  0  0
 0  0  0  0
 0  1  0  0

[:, :, 3] =
 0  0  1  0
 0  0  0  0
 0  0  0  1
 0  0  0  0

[:, :, 4] =
 0  0  0  0
 0  0  1  0
 0  0  0  0
 0  0  0  1

In [6]:
A = rand(2, 2)
B = rand(2, 2)
C = zeros(2, 2)

for k in LinearIndices(C)
    C[k] = sum([T[i,j,k] * A[i] * B[j] for i=1:4, j=1:4])
end

C == A * B

true

### 6 Implementing multiplication via the CPD

In [7]:
function multiply(A::AbstractMatrix, B::AbstractMatrix, U::AbstractMatrix, V::AbstractMatrix, W::AbstractMatrix)
    # check arguments
    n, n2 = size(A)
    n3, n4 = size(B)
    if n != n2 || n != n3 || n != n4
        throw(ArgumentError("Matrices to be multiplied need to be square with the same dimensions."))
    end
    nu, r = size(U)
    nv, rv = size(V)
    nw, rw = size(W)
    if r != rv || r != rw
        throw(ArgumentError("Rank dimension of CPD matrices do not match."))
    end
    if nu != n^2 || nv != n^2 || nw != n^2
        throw(ArgumentError("CPD matrices need to be of dimensions $(n^2) × $r"))
    end

    # initialize matrix
    T = promote_type(eltype(A), eltype(B))
    C = similar(A, T)

    # essential multiplications
    m = zeros(T, r)
    for α = 1:r
        UA = sum([U[i,α] * A[i] for i in LinearIndices(A)])
        VB = sum([V[i,α] * B[i] for i in LinearIndices(B)])
        m[α] = UA * VB
    end
    
    # non-essential multiplications
    for i in LinearIndices(C)
        C[i] = sum([W[i,α] * m[α] for α = 1:r])
    end

    return C
end

multiply (generic function with 1 method)

In [8]:
A = rand(2, 2)
B = rand(2, 2)

multiply(A, B, U, V, W) == A * B

true

### 7 Interpreting the Strassen algorithm in terms of CPD

Strassen algorithm:

$$ \begin{pmatrix} c_1 & c_3 \\ c_2 & c_4 \end{pmatrix} = \begin{pmatrix} m_1 + m_4 - m_5 + m_7 & m_3 + m_5 \\ m_2 + m_4 & m_1 - m_2 + m_3 + m_6 \end{pmatrix} $$

where the $m_i$ are the following essential multiplications

$$\begin{align*}
& m_1 = ( a_1 + a_4 )( b_1 + b_4 ) \\
& m_2 = ( a_2 + a_4 ) b_1 \\
& m_3 = a_1 ( b_3 - b_4 ) \\
& m_4 = a_4 ( b_2 - b_1 ) \\
& m_5 = ( a_1 + a_3 ) b_4 \\
& m_6 = ( a_2 - a_1 )( b_1 + b_3 ) \\
& m_7 = ( a_3 - a_4 )( b_2 + b_4 ) \\
\end{align*}$$

Thus we get the following rank-7 CPD of the multiplication tensor $T$:

In [9]:
Uh = [1 0 1 0 1 -1 0; 0 1 0 0 0 1 0; 0 0 0 0 1 0 1; 1 1 0 1 0 0 -1]
Vh = [1 1 0 -1 0 1 0; 0 0 0 1 0 0 1; 0 0 1 0 0 1 0; 1 0 -1 0 1 0 1]
Wh = [1 0 0 1 -1 0 1; 0 1 0 1 0 0 0; 0 0 1 0 1 0 0; 1 -1 1 0 0 1 0]

evalCPD(Uh, Vh, Wh) == T

true

In [10]:
A = rand(2, 2)
B = rand(2, 2)

multiply(A, B, Uh, Vh, Wh) - A * B

2×2 Matrix{Float64}:
 -5.55112e-17   0.0
  2.77556e-17  -6.93889e-17